In [50]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/MyDrive/MSc/deep_learning"
# !dir "data/melgrams"

In [51]:
#setting input files and folder-dirs



CLASSES = ['bouncy','tekno','warzone','industrial','non-techno-drop']
# REMOVED: 'psytech',

# csv_file_path = "/content/drive/MyDrive/MSc/deep_learning/labels_final.csv"
# tracks_dir = "/content/drive/MyDrive/MSc/deep_learning/tracks_final"
# clips_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/clips"

# norm_melgrams_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/norm_melgrams_dir"
# low_melgrams_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/low_melgrams"
# melgrams_dir = norm_melgrams_dir # low_melgrams_dir


csv_file_path = "data/labels_final.csv"
tracks_dir = "data/tracks_final"
clips_dir = "data/clips"

norm_melgrams_dir = "data/melgrams"
low_melgrams_dir = "data/low_melgrams"
melgrams_dir = norm_melgrams_dir # low_melgrams_dir

import os
os.makedirs(clips_dir, exist_ok=True)
os.makedirs(norm_melgrams_dir, exist_ok=True)
os.makedirs(low_melgrams_dir, exist_ok=True)

SEED = 42

# import random, numpy as np
# from tensorflow.keras.utils import set_random_seed
# random.seed(SEED)
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# set_random_seed(SEED)


In [52]:
# # sample-clipper > clips are within "deep_learning/clips" (including non-techno-drop)



# import os, pandas as pd, librosa, soundfile as sf

# df = pd.read_csv(csv_file_path)
# total_clips = len(df)
# processed = 0

# # convert m:ss to float seconds

# def ts_to_sec(ts: str) -> float:
#     if ":" in ts:
#         m, s = map(int, ts.split(":"))
#         return m * 60 + s
#     else:
#         return float(ts)

# # iterating rows and create clips

# for idx, row in df.iterrows():
#     spot_id    = str(row["spot_id"])
#     sample_id  = str(row["sample_id"])
#     clip_start = ts_to_sec(str(row["clip_start"]))

#     # skip if clip exists
#     dst = os.path.join(clips_dir, f"{sample_id}.mp3")
#     if os.path.exists(dst):
#         print(f"[SKIP] Clip exists: {sample_id}.mp3")
#         processed += 1
#         continue

#     duration = 8

#     src = os.path.join(tracks_dir, f"{spot_id}.mp3")

#     if not os.path.exists(src):
#         print(f"[SKIP] track file not found: {src}")
#         processed += 1
#         continue

#     try:
#         y, sr = librosa.load(src, sr=None)
#         beg = int(clip_start * sr)
#         end = int((clip_start + duration) * sr)
#         clip = y[beg:end]

#         sf.write(dst, clip, sr)
#         processed += 1
#         print(f"[OK] {sample_id}.mp3 | {duration}s ({processed}/{total_clips})")

#     except Exception as e:
#         print(f"[ERR] {sample_id}: {e}")

# print(f"done {processed}/{total_clips} clips.")

In [53]:
### see how many we clips we got after all


import pandas as pd
import librosa
import soundfile as sf
import os

n_written = sum(1 for f in os.listdir(clips_dir))
print(n_written)

1772


In [54]:
### features for ML predictions with librosa



import pandas as pd
import librosa
import librosa.display
import numpy as np
import os

df_csv = pd.read_csv(csv_file_path)

features = []
labels = []

# function to get features with librosa
def extract_features(audio_file):
    y, sr = librosa.load(audio_file)

    # feature list
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y=y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    # getting means
    return [
        np.mean(chroma_stft),
        np.mean(rmse),
        np.mean(spec_cent),
        np.mean(spec_bw),
        np.mean(rolloff),
        np.mean(zcr),
        np.mean(mfcc[0]),
        np.mean(mfcc[1]),
        np.mean(mfcc[2]),
        np.mean(mfcc[3]),
        np.mean(mfcc[4]),
        np.mean(mfcc[5]),
        np.mean(mfcc[6]),

    ]

# going through the clips
for filename in os.listdir(clips_dir):
    if filename.endswith(".mp3"):
        clip_path = os.path.join(clips_dir, filename)

        # getting features
        clip_features = extract_features(clip_path)
        features.append(clip_features)

        # getting the name as well, to match them later
        sample_id = filename[:-4]

        # matching with the labels from the csv
        try:
            row = df_csv.loc[df_csv['sample_id'] == sample_id, CLASSES].iloc[0].tolist()
            # convert duration → binary flag (misc)
            misc_val   = row[-1]
            misc_label = 1 if pd.notna(misc_val) and int(misc_val) > 0 else 0
            row[-1]    = misc_label

        except IndexError:
            print(f"[WARN] no label for {sample_id}")
            row = [np.nan]*8

        labels.append(row)


# now we will get it all into a dataframe to work with it in the svm
feature_names = [
    'chroma_stft', 'rmse', 'spec_cent', 'spec_bw', 'rolloff', 'zcr',
    'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7'
]
df_features = pd.DataFrame(features, columns=feature_names)
df_labels = pd.DataFrame(labels, columns=CLASSES)
df_final = pd.concat([df_features, df_labels], axis=1)

# saving it
df_final.to_csv("data/clip_features_and_labels_2.csv", index=False)

c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\libro

In [55]:
# !pip install pyAudioAnalysis eyed3 pydub

In [56]:
### features for ML predictions with pyAudioAnalysis



import pandas as pd
from pyAudioAnalysis.audioBasicIO import read_audio_file
from pyAudioAnalysis.ShortTermFeatures import feature_extraction
import numpy as np
import os

df_csv = pd.read_csv(csv_file_path)

features = []
labels = []

# extract features with pyAudioAnalysis
def extract_features(audio_file):
    try:
        [fs, x] = read_audio_file(audio_file)

        # extracting features
        [feats, _] = feature_extraction(
            x, fs, 0.050 * fs, 0.025 * fs)

        # feature mapping as before
        return [
            np.mean(feats[13:25]),
            np.mean(feats[1]),
            np.mean(feats[3]),
            np.mean(feats[4]),
            np.mean(feats[7]),
            np.mean(feats[0]),
            *[np.mean(feats[8+i]) for i in range(5)],
            np.mean(feats[8:13]),
            np.mean(feats[8:13])
        ]
    except Exception as e:
        print(f"Error processing {audio_file}: {str(e)}")
        return None

for filename in os.listdir(clips_dir):
    if filename.endswith(".mp3"):
        clip_path = os.path.join(clips_dir, filename)

        # get the features
        try:
            clip_features = extract_features(clip_path)
            features.append(clip_features)
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue

        # match labels from csv
        sample_id = filename[:-4]
        try:
            clip_labels = df_csv.loc[
                df_csv['sample_id'] == sample_id, CLASSES].values.tolist()[0]
        except IndexError:
            print(f"Warning: No label found for sample_id: {sample_id}")
            clip_labels = [np.nan] * 8

        labels.append(clip_labels)

# creating the dataframes
feature_names = [
    'chroma_stft', 'rmse', 'spec_cent', 'spec_bw', 'rolloff', 'zcr',
    'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7'
]
df_features = pd.DataFrame(features, columns=feature_names)
df_labels = pd.DataFrame(labels, columns=CLASSES)
df_final = pd.concat([df_features, df_labels], axis=1)

# saving
df_final.to_csv("data/clip_features_and_labels_pyAudioAnalysis.csv", index=False)

In [57]:
### filtering rows



import pandas as pd

# loading the df from before
df = pd.read_csv("data/clip_features_and_labels_2.csv") # << Librosa
# df = pd.read_csv("data/clip_features_and_labels_pyAudioAnalysis.csv") # << pyAudioAnalysis

df['label_sum'] = df[CLASSES].sum(axis=1)
filtered_df = df[df['label_sum'] >= 1]
filtered_df = filtered_df.drop(columns=['label_sum'])

In [58]:
### prep-split & predictions based on features



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# features and labels
X = filtered_df[['chroma_stft', 'rmse', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7']]
y = filtered_df[CLASSES]

# splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# scaling for svm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
# purely for debugging during testing



import pandas as pd

# checking for missing values
missing_labels = y_train.isnull().sum()
print("Missing values in labels (y_train):\n", missing_labels)

X_train_df = pd.DataFrame(X_train, columns=['chroma_stft', 'rmse', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7'])
missing_features = X_train_df.isnull().sum()
print("\nMissing values in features (X_train):\n", missing_features)

Missing values in labels (y_train):
 bouncy             0
tekno              0
warzone            0
industrial         0
non-techno-drop    0
dtype: int64

Missing values in features (X_train):
 chroma_stft    0
rmse           0
spec_cent      0
spec_bw        0
rolloff        0
zcr            0
mfcc1          0
mfcc2          0
mfcc3          0
mfcc4          0
mfcc5          0
mfcc6          0
mfcc7          0
dtype: int64


In [60]:
### new SVM for single-label



from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report

svm = OneVsRestClassifier(
        SVC(kernel='rbf', probability=False, class_weight='balanced'))

svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

for i, label in enumerate(y.columns):
    print(f"\nMetrics for {label}:")
    print("Accuracy: ", accuracy_score(y_test.iloc[:, i], y_pred[:, i]))
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

print(classification_report(y_test, y_pred, target_names=CLASSES))




Metrics for bouncy:
Accuracy:  0.8544061302681992
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       213
           1       0.57      0.83      0.68        48

    accuracy                           0.85       261
   macro avg       0.76      0.85      0.79       261
weighted avg       0.89      0.85      0.86       261


Metrics for tekno:
Accuracy:  0.8275862068965517
              precision    recall  f1-score   support

           0       0.94      0.84      0.89       208
           1       0.55      0.79      0.65        53

    accuracy                           0.83       261
   macro avg       0.75      0.81      0.77       261
weighted avg       0.86      0.83      0.84       261


Metrics for warzone:
Accuracy:  0.8390804597701149
              precision    recall  f1-score   support

           0       0.96      0.84      0.89       209
           1       0.56      0.85      0.68        52

    accuracy                 

In [61]:
### new XGBoost for single-label



import pandas as pd, xgboost as xgb, numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Dy_train = np.asarray(y_train)
Dy_test  = np.asarray(y_test)

Dy_train_idx = np.argmax(Dy_train, axis=1)
Dy_test_idx  = np.argmax(Dy_test,  axis=1)

num_class = Dy_train.shape[1] 

dtrain = xgb.DMatrix(X_train, label=Dy_train_idx)
dval   = xgb.DMatrix(X_test,  label=Dy_test_idx)

params = dict(
    objective     = 'multi:softprob',
    num_class     = num_class,
    eval_metric   = 'mlogloss',
    max_depth     = 6,
    eta           = 0.1,
    subsample     = 0.9,
    colsample_bytree = 0.9
)

booster = xgb.train(params, dtrain,
                    num_boost_round=600,
                    evals=[(dtrain,'train'), (dval,'val')],
                    early_stopping_rounds=30,
                    verbose_eval=50)


y_pred = booster.predict(dval).argmax(axis=1)

#  overall multi class report
print("\n=== Overall report on XGBoost validation set ===")
print(classification_report(Dy_test_idx, y_pred, target_names=CLASSES))


[0]	train-mlogloss:1.47081	val-mlogloss:1.50340


[50]	train-mlogloss:0.17770	val-mlogloss:0.71909
[99]	train-mlogloss:0.06384	val-mlogloss:0.71683

=== Overall report on XGBoost validation set ===
                 precision    recall  f1-score   support

         bouncy       0.63      0.69      0.66        48
          tekno       0.68      0.68      0.68        53
        warzone       0.73      0.79      0.76        52
     industrial       0.76      0.64      0.70        61
non-techno-drop       0.92      0.96      0.94        47

       accuracy                           0.74       261
      macro avg       0.75      0.75      0.75       261
   weighted avg       0.74      0.74      0.74       261



In [62]:
### hyper-tunning XGBoost



param_grid_xgb = {
    'max_depth'        : [4, 6, 8, 10],
    'eta'              : [0.30, 0.15, 0.07, 0.03],
    'n_estimators'     : [200, 400, 800, 1200],
    'subsample'        : [0.7, 0.85, 1.0],
    'colsample_bytree' : [0.6, 0.8, 1.0],
    'min_child_weight' : [1, 3, 5],
    'gamma'            : [0, 0.5, 1],
    'lambda_l1'        : [0,  1.0],
    'lambda_l2'        : [0,  1.0],
    'tree_method'      : ['hist'],
}

from sklearn.model_selection import ParameterSampler
from sklearn.metrics import f1_score
search = list(ParameterSampler(param_grid_xgb, n_iter=25, random_state=SEED))

best_macro_f1 = 0
best_params   = None

for p in search:
    model = xgb.XGBClassifier(objective='multi:softprob',
                              num_class=len(CLASSES),
                              eval_metric='mlogloss',
                              early_stopping_rounds=50,
                              **p)
    model.fit(X_train, Dy_train_idx,
              eval_set=[(X_test, Dy_test_idx)],
              verbose=False)
    y_pred = model.predict(X_test)
    f1 = f1_score(Dy_test_idx, y_pred, average='macro')
    if f1 > best_macro_f1:
        best_macro_f1, best_params = f1, p

print("Best XGB  macro-F1:", best_macro_f1, "\nParams:", best_params)


c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\callback.py:386: UserWarning: [01:32:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "lambda_l1", "lambda_l2" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\callback.py:386: UserWarning: [01:32:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "lambda_l1", "lambda_l2" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\callback.py:386: UserWarning: [01:32:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "lambda_l1", "lambda_l2" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\callback.py:386: UserWarning: [01:

Best XGB  macro-F1: 0.7675351811960717 
Params: {'tree_method': 'hist', 'subsample': 0.85, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 8, 'lambda_l2': 1.0, 'lambda_l1': 1.0, 'gamma': 0.5, 'eta': 0.03, 'colsample_bytree': 0.6}


In [63]:
### *TUNED XGBoost for single-label



import pandas as pd, xgboost as xgb, numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Dy_train = np.asarray(y_train)
Dy_test  = np.asarray(y_test)

Dy_train_idx = np.argmax(Dy_train, axis=1)
Dy_test_idx  = np.argmax(Dy_test,  axis=1)

num_class = Dy_train.shape[1] 

dtrain = xgb.DMatrix(X_train, label=Dy_train_idx)
dval   = xgb.DMatrix(X_test,  label=Dy_test_idx)

params = best_params.copy()
params.pop("n_estimators", None)
params.update(dict(
    objective   = 'multi:softprob',
    num_class   = len(CLASSES),
    eval_metric = 'mlogloss'
))


booster = xgb.train(params, dtrain,
                    num_boost_round=600,
                    evals=[(dtrain,'train'), (dval,'val')],
                    early_stopping_rounds=30,
                    verbose_eval=50)


y_pred = booster.predict(dval).argmax(axis=1)

#  overall multi class report
print("for *TUNED* XGBoost")
print(classification_report(Dy_test_idx, y_pred, target_names=CLASSES))


[0]	train-mlogloss:1.56760	val-mlogloss:1.57687


c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\callback.py:386: UserWarning: [01:32:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "lambda_l1", "lambda_l2" } are not used.

  self.starting_round = model.num_boosted_rounds()


[50]	train-mlogloss:0.61683	val-mlogloss:0.94030
[100]	train-mlogloss:0.33241	val-mlogloss:0.76317
[150]	train-mlogloss:0.22592	val-mlogloss:0.71270
[200]	train-mlogloss:0.18648	val-mlogloss:0.69622
[250]	train-mlogloss:0.17249	val-mlogloss:0.69185
[300]	train-mlogloss:0.16464	val-mlogloss:0.68935
[350]	train-mlogloss:0.16056	val-mlogloss:0.68896
[351]	train-mlogloss:0.16037	val-mlogloss:0.68916
for *TUNED* XGBoost
                 precision    recall  f1-score   support

         bouncy       0.66      0.65      0.65        48
          tekno       0.67      0.72      0.69        53
        warzone       0.75      0.83      0.79        52
     industrial       0.82      0.67      0.74        61
non-techno-drop       0.92      0.98      0.95        47

       accuracy                           0.76       261
      macro avg       0.76      0.77      0.76       261
   weighted avg       0.76      0.76      0.76       261



In [64]:
### new LightGBM for ML



import lightgbm as lgb

y_train_idx = np.argmax(y_train, axis=1)
y_test_idx  = np.argmax(y_test, axis=1)

train_set = lgb.Dataset(X_train, label=y_train_idx)
val_set   = lgb.Dataset(X_test,   label=y_test_idx, reference=train_set)

params_lgb = dict(
    objective        = 'multiclass',
    num_class        = len(CLASSES),
    metric           = 'multi_logloss',
    learning_rate    = 0.05,
    num_leaves       = 31,
    feature_fraction = 0.9,
    bagging_fraction = 0.9,
    bagging_freq     = 5,
    seed             = SEED,
    verbosity        = -1 
)

lgbm = lgb.train(
        params_lgb,
        train_set,
        num_boost_round = 2000,
        valid_sets      = [train_set, val_set],
        valid_names     = ['train',  'val'],
        callbacks       = [lgb.early_stopping(stopping_rounds=100,
                                             verbose=True),
                           lgb.log_evaluation(period=100)]
)

# prediction
y_pred = np.argmax(lgbm.predict(X_test), axis=1)

print(classification_report(y_test_idx, y_pred, target_names=CLASSES))


Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.0463771	val's multi_logloss: 0.728026
Early stopping, best iteration is:
[69]	train's multi_logloss: 0.125694	val's multi_logloss: 0.710398
                 precision    recall  f1-score   support

         bouncy       0.72      0.65      0.68        48
          tekno       0.66      0.70      0.68        53
        warzone       0.74      0.83      0.78        52
     industrial       0.77      0.67      0.72        61
non-techno-drop       0.90      0.98      0.94        47

       accuracy                           0.76       261
      macro avg       0.76      0.76      0.76       261
   weighted avg       0.76      0.76      0.76       261



In [65]:
### hyper-tunning LightGBM



from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import f1_score

param_grid = {
    'max_depth'        : [4, 6, 8, -1],
    'learning_rate'    : [0.20, 0.10, 0.05],
    'n_estimators'     : [400, 800, 1600],
    'feature_fraction' : [0.7, 0.85, 1.0],
    'bagging_fraction' : [0.7, 0.85, 1.0],
    'bagging_freq'     : [0, 5],
    'min_child_samples': [10, 25, 50],
    'min_split_gain'   : [0, 0.2],
    'reg_alpha'        : [0, 0.5],
    'reg_lambda'       : [0, 1.0]
}

search = ParameterSampler(param_grid, n_iter=25, random_state=SEED)

best_f1, best_params = 0, None
for p in search:
    clf = LGBMClassifier(objective='multiclass',
                         num_class=len(CLASSES),
                         **p)

    clf.fit(
        X_train, y_train_idx,
        eval_set=[(X_test, y_test_idx)],
        eval_metric='multi_logloss',
        callbacks=[early_stopping(50, verbose=False),
                   log_evaluation(period=0)]
    )

    f1 = f1_score(y_test_idx, clf.predict(X_test), average='macro')
    if f1 > best_f1:
        best_f1, best_params = f1, p

print("Best LGBM macro-F1:", best_f1)
print("Parameters:", best_params)


c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X d

Best LGBM macro-F1: 0.7680522864037456
Parameters: {'reg_lambda': 0, 'reg_alpha': 0, 'n_estimators': 400, 'min_split_gain': 0.2, 'min_child_samples': 10, 'max_depth': 8, 'learning_rate': 0.05, 'feature_fraction': 0.85, 'bagging_freq': 0, 'bagging_fraction': 1.0}


c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [66]:
### *TUNED LightGBM



import lightgbm as lgb

train_set = lgb.Dataset(X_train, label=y_train_idx)
val_set   = lgb.Dataset(X_test,   label=y_test_idx, reference=train_set)

params_lgb = best_params.copy()
params_lgb.update({
    'objective': 'multiclass',
    'num_class': len(CLASSES),
    'metric': 'multi_logloss',
    'seed': SEED,
    'verbosity': -1 
})

num_boost_round = params_lgb.pop('n_estimators')

#  training with callbacks
lgbm = lgb.train(
    params_lgb,
    train_set,
    num_boost_round = 1600,
    valid_sets      = [train_set, val_set],
    valid_names     = ['train', 'val'],
    callbacks       = [
        lgb.early_stopping(stopping_rounds=100, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)

# prediction
y_pred = np.argmax(lgbm.predict(X_test), axis=1)

print("for *TUNED* LightGBM")
print(classification_report(y_test_idx, y_pred, target_names=CLASSES))


Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.0704831	val's multi_logloss: 0.714505
Early stopping, best iteration is:
[63]	train's multi_logloss: 0.116869	val's multi_logloss: 0.706085
for *TUNED* LightGBM
                 precision    recall  f1-score   support

         bouncy       0.70      0.69      0.69        48
          tekno       0.70      0.70      0.70        53
        warzone       0.74      0.81      0.77        52
     industrial       0.76      0.67      0.71        61
non-techno-drop       0.94      1.00      0.97        47

       accuracy                           0.77       261
      macro avg       0.77      0.77      0.77       261
   weighted avg       0.76      0.77      0.76       261



In [67]:
### soft-vote stacking



# xgboost
params_xgb = params.copy()
params_xgb.update({
    'objective': 'multi:softprob',
    'num_class': len(CLASSES),
    'eval_metric': 'mlogloss',
    'seed': SEED
})

dtrain_xgb = xgb.DMatrix(X_train, label=y_train_idx)
dtest_xgb  = xgb.DMatrix(X_test)

booster_xgb = xgb.train(params_xgb, dtrain_xgb, num_boost_round=200)

# lgbm
params_lgbm = params_lgb.copy()  
params_lgbm.update({
    'objective': 'multiclass',
    'num_class': len(CLASSES),
    'metric': 'multi_logloss',
    'seed': SEED,
    'verbosity': -1  
})

train_set_lgb = lgb.Dataset(X_train, label=y_train_idx)
booster_lgb = lgb.train(params_lgbm,
                        train_set_lgb,
                        num_boost_round=400)

# pedict 
proba_xgb = booster_xgb.predict(dtest_xgb)
proba_lgb = booster_lgb.predict(X_test)

# soft-vote stacking
alpha = 0.5 
proba_stack = alpha * proba_xgb + (1 - alpha) * proba_lgb
y_pred_stack = proba_stack.argmax(axis=1)

print(classification_report(y_test_idx, y_pred_stack, target_names=CLASSES))


c:\Users\me\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [01:32:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "lambda_l1", "lambda_l2" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


                 precision    recall  f1-score   support

         bouncy       0.71      0.67      0.69        48
          tekno       0.66      0.70      0.68        53
        warzone       0.77      0.83      0.80        52
     industrial       0.78      0.69      0.73        61
non-techno-drop       0.92      0.98      0.95        47

       accuracy                           0.77       261
      macro avg       0.77      0.77      0.77       261
   weighted avg       0.77      0.77      0.76       261



In [68]:
# ### creating normal melgrams



# import librosa
# import librosa.display
# import matplotlib.pyplot as plt
# import numpy as np

# SR      = 22_050
# N_MELS  = 128
# F_CUT   = 8_000

# def save_melgram(input_path, output_path):

#     # loading the audio file
#     y, sr = librosa.load(input_path, sr=22_050)

#     # generating the mel spectograms
#     S = librosa.feature.melspectrogram(y=y, sr=sr,
#                                        n_fft=1024,
#                                        hop_length=512,
#                                        n_mels=128,
#                                        fmin=20,
#                                        fmax=11_000)
#     S_db = librosa.power_to_db(S, ref=np.max)

#     # plotting the spectograms
#     plt.figure(figsize=(3, 3))
#     librosa.display.specshow(S_db, sr=sr, hop_length=512,
#                              x_axis='time', y_axis='mel',
#                              fmax=11_000, cmap='gray_r')   # grayscale

#     plt.axis('off')  # removing the axis
#     plt.tight_layout(pad=0)

#     # saving
#     plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)
#     plt.close()

# # going through the clips, to make the spectograms
# for filename in os.listdir(clips_dir):
#     if filename.endswith(".mp3"):
#         clip_path = os.path.join(clips_dir, filename)
#         melgram_path = os.path.join(melgrams_dir, filename[:-4] + ".png")
#         save_melgram(clip_path, melgram_path)
#         print(f"Melgram saved for {filename}")


In [69]:
# ### creating melgrams with low-pass filter



# import librosa
# import librosa.display
# import matplotlib.pyplot as plt
# import numpy as np

# SR      = 22_050
# N_MELS  = 128
# F_CUT   = 2_000        # low-pass corner

# def save_melgram(input_path, output_path):

#     # loading the audio file
#     y, _ = librosa.load(input_path, sr=SR)

#     # generating the mel spectograms

#     M = librosa.feature.melspectrogram(
#             y=y, sr=SR,
#             n_fft=1024,
#             hop_length=512,
#             n_mels=N_MELS,
#             fmin=20,
#             fmax=F_CUT)

#     M_db = librosa.power_to_db(M, ref=np.max).astype(np.float32)

#     # plot PNG > grayscale
#     plt.figure(figsize=(3, 3))
#     librosa.display.specshow(M_db, sr=SR,
#                              hop_length=512,
#                              x_axis='time', y_axis='mel',
#                              fmax=F_CUT,
#                              cmap='gray_r')
#     plt.axis('off')
#     plt.tight_layout(pad=0)
#     plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)
#     plt.close()

# # going through the clips, to make the spectograms
# for filename in os.listdir(clips_dir):
#     if filename.endswith(".mp3"):
#         clip_path = os.path.join(clips_dir, filename)
#         low_melgram_path = os.path.join(low_melgrams_dir, filename[:-4] + ".png")
#         save_melgram(clip_path, low_melgram_path)
#         print(f"Melgram saved for {filename}")


In [70]:
### see how many we melgrams we got after all
#1772 ?

import pandas as pd
import librosa
import soundfile as sf
import os

n_written = sum(1 for f in os.listdir(low_melgrams_dir))
print(low_melgrams_dir)
print(n_written)

data/low_melgrams
1772


In [71]:
# ### connect to gdrive


# from google.colab import drive
# drive.mount('/content/drive')

# !ls "/content/drive/MyDrive/MSc/deep_learning/apostolis/"

In [72]:
### setting input files and folder-dirs



CLASSES = ['bouncy','tekno','warzone','industrial','non-techno-drop']
# REMOVED: 'psytech',

# csv_file_path = "/content/drive/MyDrive/MSc/deep_learning/labels_final.csv"
# tracks_dir = "/content/drive/MyDrive/MSc/deep_learning/tracks_final"
# clips_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/clips"

# norm_melgrams_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/norm_melgrams_dir"
# low_melgrams_dir = "/content/drive/MyDrive/MSc/deep_learning/apostolis/low_melgrams"
# melgrams_dir = norm_melgrams_dir # low_melgrams_dir

csv_file_path = "data/labels_final.csv"
tracks_dir = "data/tracks_final"
clips_dir = "data/clips"

norm_melgrams_dir = "data/melgrams"
low_melgrams_dir = "data/low_melgrams"
melgrams_dir = norm_melgrams_dir # low_melgrams_dir

import os
os.makedirs(clips_dir, exist_ok=True)
os.makedirs(norm_melgrams_dir, exist_ok=True)
os.makedirs(low_melgrams_dir, exist_ok=True)

SEED = 42

# import random, numpy as np
# from tensorflow.keras.utils import set_random_seed
# random.seed(SEED)
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# set_random_seed(SEED)


In [73]:
### mapping labels to melgrams



import pandas as pd
import os

# loading the csv with the labels
df_csv = pd.read_csv(csv_file_path)

# creating a dictionary to keep the mapping
melgram_label_mapping = {}

# going through the csv
for index, row in df_csv.iterrows():
    sample_id = str(row['sample_id'])
    melgram_filename = sample_id + ".png"

    labels = row[CLASSES].values.tolist()

    # check to ensure at least one label is 1
    if any(label == 1 for label in labels):
        melgram_label_mapping[melgram_filename] = labels


In [74]:
# prep-split for train/val/test



# build X, y + a 'groups' list with song IDs, to ensure no same-song clips fall in different train/val/test sets !

from pathlib import Path
import cv2, numpy as np, pandas as pd, os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

image_size = (128, 128)
input_shape = (128, 128, 1)
labels_list = CLASSES

df_csv = pd.read_csv(csv_file_path).set_index('sample_id')

def list_to_idx(vec):
    idxs = [i for i, v in enumerate(vec) if v == 1]
    return idxs[0] if len(idxs) == 1 else None

X, y_int, groups = [], [], []

for fname, vec in melgram_label_mapping.items():
    class_idx = list_to_idx(vec)
    if class_idx is None:
        continue

    png_path = os.path.join(melgrams_dir, fname)
    if not os.path.exists(png_path):
        continue

    # load & preprocess to (128,128,1)
    img = cv2.imread(png_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, -1).astype(np.float32) / 255.0

    X.append(img)
    y_int.append(class_idx)

    sample_id = Path(fname).stem
    groups.append(df_csv.loc[sample_id, 'spot_id'])

X       = np.array(X,       dtype=np.float32)
y_int   = np.array(y_int,  dtype=np.int32)
groups  = np.array(groups)

print("Loaded tensors:", X.shape)

# first split, 70% train, 30% temp (val+test)

sgkf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=SEED)
train_idx, temp_idx = next(sgkf.split(X, y_int, groups=groups))

# 2nd split, 50/50 on temp → 15% val, 15% test

sgkf_temp = StratifiedGroupKFold(n_splits=2, shuffle=True, random_state=SEED)
val_idx, test_idx = next(sgkf_temp.split(X[temp_idx], y_int[temp_idx], groups=groups[temp_idx]))

X_train, y_train = X[train_idx], y_int[train_idx]
X_val, y_val = X[temp_idx][val_idx], y_int[temp_idx][val_idx]
X_test, y_test = X[temp_idx][test_idx], y_int[temp_idx][test_idx]

print(f"\nTrain clips : {len(X_train)}")
print(f"Valid clips : {len(X_val)}")
print(f"Test  clips : {len(X_test)}")

print(f"\nTrain labels : {len(y_train)}")
print(f"Valid labels : {len(y_val)}")
print(f"Test  labels : {len(y_test)}")

Loaded tensors: (1305, 128, 128, 1)

Train clips : 872
Valid clips : 207
Test  clips : 226

Train labels : 872
Valid labels : 207
Test  labels : 226


In [75]:
# !pip install tensorflow

In [76]:
### train-set check + build metric-printers



print("Unique labels in y_train:", np.unique(y_train))
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Sample y_train:", y_train[0])
print("y_train dtype:", y_train.dtype)

from sklearn.metrics import f1_score, classification_report, confusion_matrix
import numpy as np

# evaluating the model on the validation & the test set

def show_val_results(history, X_val, y_val, class_names):

    # epoch with the lowest val-loss

    best = np.argmin(history.history['val_loss'])
    print(f"\nBest epoch = {best+1}")
    print(f"  train_acc {history.history['accuracy'][best]:.4f}"
          f" | train_loss {history.history['loss'][best]:.4f}")
    print(f"  val_acc   {history.history['val_accuracy'][best]:.4f}"
          f" | val_loss  {history.history['val_loss'][best]:.4f}")

    y_pred = history.model.predict(X_val, verbose=0).argmax(axis=1)
    print(f"\nWeighted F1 (val) : {f1_score(y_val, y_pred, average='weighted'):.4f}")
    print(classification_report(y_val, y_pred, target_names=class_names))
    print("Confusion matrix (val):\n", confusion_matrix(y_val, y_pred))


def show_test_results(modelo, X_test, y_test, class_names):

    y_pred = modelo.predict(X_test, verbose=0).argmax(axis=1)

    print("\n=== TEST-SET RESULTS ===")
    print(f"Weighted F1 (test): {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(classification_report(y_test, y_pred, target_names=class_names))
    print("Confusion matrix (test):\n", confusion_matrix(y_test, y_pred))


Unique labels in y_train: [0 1 2 3 4]
X_train shape: (872, 128, 128, 1)
y_train shape: (872,)
Sample y_train: 3
y_train dtype: int32


In [77]:
### check & verify class distributions


def print_class_dist(y, name):
    unique, counts = np.unique(y, return_counts=True)
    print(f"{name} class counts:", dict(zip(labels_list, counts)))

print_class_dist(y_train, "Train")
print_class_dist(y_val, "Val")
print_class_dist(y_test, "Test")

Train class counts: {'bouncy': np.int64(184), 'tekno': np.int64(183), 'warzone': np.int64(177), 'industrial': np.int64(170), 'non-techno-drop': np.int64(158)}
Val class counts: {'bouncy': np.int64(35), 'tekno': np.int64(30), 'warzone': np.int64(45), 'industrial': np.int64(47), 'non-techno-drop': np.int64(50)}
Test class counts: {'bouncy': np.int64(42), 'tekno': np.int64(46), 'warzone': np.int64(44), 'industrial': np.int64(42), 'non-techno-drop': np.int64(52)}


In [78]:
### class weight to handle class imbalance, especially in the val & test sets



from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [79]:
### main CNN construction



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

BATCH = 64

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
    MaxPooling2D(2,2), # MAYBE (2,1) OR (1,2) ?

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(labels_list), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

# model.summary()

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history1 = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=BATCH,
    shuffle=True,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=2
)

show_val_results(history1, X_val, y_val, CLASSES)
show_test_results(model,   X_test, y_test, CLASSES)



#################



# results with main-CNN for normal melgrams after 5 runs, WITHOUT fixed tf.seed

# run 1
# Best epoch = 14
#   train_acc 0.8647 | train_loss 0.3721
#   val_acc   0.8068 | val_loss  0.6143
# Weighted F1 (val) : 0.8050
# Weighted F1 (test): 0.7621

# run 2
# Best epoch = 22
#   train_acc 0.9472 | train_loss 0.1642
#   val_acc   0.8454 | val_loss  0.5026
# Weighted F1 (val) : 0.8448
# Weighted F1 (test): 0.8101

# run 3
# Best epoch = 14
#   train_acc 0.8784 | train_loss 0.3704
#   val_acc   0.8502 | val_loss  0.5496
# Weighted F1 (val) : 0.8482
# Weighted F1 (test): 0.7641

# run 4
# Best epoch = 23
#   train_acc 0.9025 | train_loss 0.2604
#   val_acc   0.8213 | val_loss  0.5751
# Weighted F1 (val) : 0.8134
# Weighted F1 (test): 0.7938

# run 5
# Best epoch = 16
#   train_acc 0.9071 | train_loss 0.2971
#   val_acc   0.8696 | val_loss  0.5438
# Weighted F1 (val) : 0.8668
# Weighted F1 (test): 0.7904



C:\Users\me\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
14/14 - 8s - 606ms/step - accuracy: 0.2271 - loss: 1.6603 - val_accuracy: 0.1691 - val_loss: 1.6154
Epoch 2/50
14/14 - 7s - 487ms/step - accuracy: 0.3142 - loss: 1.5432 - val_accuracy: 0.4058 - val_loss: 1.4294
Epoch 3/50
14/14 - 7s - 495ms/step - accuracy: 0.4243 - loss: 1.3503 - val_accuracy: 0.5942 - val_loss: 1.1910
Epoch 4/50
14/14 - 8s - 545ms/step - accuracy: 0.5057 - loss: 1.2455 - val_accuracy: 0.5845 - val_loss: 1.1482
Epoch 5/50
14/14 - 8s - 567ms/step - accuracy: 0.5906 - loss: 1.1012 - val_accuracy: 0.6377 - val_loss: 1.0342
Epoch 6/50
14/14 - 7s - 489ms/step - accuracy: 0.6433 - loss: 0.9930 - val_accuracy: 0.6473 - val_loss: 0.9685
Epoch 7/50
14/14 - 7s - 494ms/step - accuracy: 0.6525 - loss: 0.8847 - val_accuracy: 0.6860 - val_loss: 0.9092
Epoch 8/50
14/14 - 7s - 476ms/step - accuracy: 0.7064 - loss: 0.8247 - val_accuracy: 0.7198 - val_loss: 0.8456
Epoch 9/50
14/14 - 7s - 484ms/step - accuracy: 0.7603 - loss: 0.7005 - val_accuracy: 0.7391 - val_loss: 0.7190
E

In [80]:
### new CNN Dropout>Flatten (2,2) ###



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# import tensorflow as tf
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

BATCH = 64

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(CLASSES), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

historyD22 = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=BATCH,
    shuffle=True,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=2
)

show_val_results(historyD22, X_val, y_val, CLASSES)
show_test_results(model,   X_test, y_test, CLASSES)





#################

# results with new-CNN for normal melgrams after 5 runs, WITHOUT fixed tf.seed

# run 1
# Best epoch = 19
#   train_acc 0.8784 | train_loss 0.3446
#   val_acc   0.7729 | val_loss  0.6463
# Weighted F1 (val) : 0.7778
# Weighted F1 (test): 0.7538

# run 2
# Best epoch = 21
#   train_acc 0.8911 | train_loss 0.3020
#   val_acc   0.8019 | val_loss  0.6091
# Weighted F1 (val) : 0.8000
# Weighted F1 (test): 0.7799

# run 3
# Best epoch = 31
#   train_acc 0.9323 | train_loss 0.1961
#   val_acc   0.7971 | val_loss  0.6636
# Weighted F1 (val) : 0.7954
# Weighted F1 (test): 0.7888

# run 4
# Best epoch = 34
#   train_acc 0.9656 | train_loss 0.1135
#   val_acc   0.7874 | val_loss  0.6383
# Weighted F1 (val) : 0.7837
# Weighted F1 (test): 0.7956

# run 5
# Best epoch = 17
#   train_acc 0.8532 | train_loss 0.4371
#   val_acc   0.7440 | val_loss  0.7363
# Weighted F1 (val) : 0.7460
# Weighted F1 (test): 0.7390


Epoch 1/50


C:\Users\me\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 - 9s - 638ms/step - accuracy: 0.2156 - loss: 1.6491 - val_accuracy: 0.1691 - val_loss: 1.6048
Epoch 2/50
14/14 - 7s - 492ms/step - accuracy: 0.2557 - loss: 1.5814 - val_accuracy: 0.4106 - val_loss: 1.5598
Epoch 3/50
14/14 - 7s - 466ms/step - accuracy: 0.3647 - loss: 1.4739 - val_accuracy: 0.4300 - val_loss: 1.3819
Epoch 4/50
14/14 - 6s - 450ms/step - accuracy: 0.4025 - loss: 1.4000 - val_accuracy: 0.4589 - val_loss: 1.3706
Epoch 5/50
14/14 - 6s - 448ms/step - accuracy: 0.5344 - loss: 1.2225 - val_accuracy: 0.5072 - val_loss: 1.3086
Epoch 6/50
14/14 - 6s - 456ms/step - accuracy: 0.5241 - loss: 1.1788 - val_accuracy: 0.4976 - val_loss: 1.2101
Epoch 7/50
14/14 - 6s - 460ms/step - accuracy: 0.6330 - loss: 1.0466 - val_accuracy: 0.6570 - val_loss: 1.0564
Epoch 8/50
14/14 - 6s - 455ms/step - accuracy: 0.6571 - loss: 0.9567 - val_accuracy: 0.6184 - val_loss: 1.0543
Epoch 9/50
14/14 - 7s - 478ms/step - accuracy: 0.6892 - loss: 0.9157 - val_accuracy: 0.7101 - val_loss: 0.9384
Epoch 10/50


In [ ]:
### previous-ultimate model?! not performing so well anymore
### >> focus on frequencies-bands rather than time/rhythm



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    MaxPooling2D(1,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(CLASSES), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # for integer labels
    metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

historyD12 = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=BATCH,
    shuffle=True,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=2          # prints: loss, acc, val_loss, val_acc each epoch
)

show_val_results(historyD12, X_val, y_val, CLASSES)
show_test_results(model,   X_test, y_test, CLASSES)


# low_melgrams
# Best epoch training: accuracy 0.8538, loss 0.4033
# Best epoch validation accuracy: 0.7921, loss 0.6513



#################

# results for normal melgrams , WITHOUT fixed tf.seed

# Best epoch = 24
#   train_acc 0.8314 | train_loss 0.4636
#   val_acc   0.7585 | val_loss  0.7646
# Weighted F1 (val) : 0.7536
# Weighted F1 (test): 0.7448

# Best epoch = 31
#   train_acc 0.8956 | train_loss 0.3021
#   val_acc   0.7681 | val_loss  0.7816
# Weighted F1 (val) : 0.7725
# Weighted F1 (test): 0.6985

# Best epoch = 29
#   train_acc 0.9312 | train_loss 0.2109
#   val_acc   0.7488 | val_loss  0.8407
# Weighted F1 (val) : 0.7499
# Weighted F1 (test): 0.6917


Epoch 1/50


C:\Users\me\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 - 8s - 537ms/step - accuracy: 0.1858 - loss: 1.7506 - val_accuracy: 0.1691 - val_loss: 1.6090
Epoch 2/50
14/14 - 6s - 425ms/step - accuracy: 0.2133 - loss: 1.6075 - val_accuracy: 0.1691 - val_loss: 1.6200
Epoch 3/50
14/14 - 6s - 420ms/step - accuracy: 0.2546 - loss: 1.5889 - val_accuracy: 0.1691 - val_loss: 1.6028
Epoch 4/50
14/14 - 8s - 594ms/step - accuracy: 0.3005 - loss: 1.5749 - val_accuracy: 0.2560 - val_loss: 1.5498
Epoch 5/50
14/14 - 7s - 521ms/step - accuracy: 0.2810 - loss: 1.5360 - val_accuracy: 0.2995 - val_loss: 1.5128
Epoch 6/50
14/14 - 7s - 504ms/step - accuracy: 0.4128 - loss: 1.4120 - val_accuracy: 0.4831 - val_loss: 1.4279
Epoch 7/50
14/14 - 6s - 458ms/step - accuracy: 0.5011 - loss: 1.2749 - val_accuracy: 0.5217 - val_loss: 1.2525
Epoch 8/50
14/14 - 6s - 440ms/step - accuracy: 0.5872 - loss: 1.1024 - val_accuracy: 0.4251 - val_loss: 1.2568
Epoch 9/50
14/14 - 6s - 423ms/step - accuracy: 0.5940 - loss: 1.0540 - val_accuracy: 0.5266 - val_loss: 1.1664
Epoch 10/50


In [82]:
### rebuilding the best model so far



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

BATCH = 64

final_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1), padding='same'),
    MaxPooling2D(2,2), # MAYBE (2,1) OR (1,2) ?

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(labels_list), activation='softmax')
])

final_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

final_history = final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=BATCH,
    shuffle=True,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=2          # prints: loss, acc, val_loss, val_acc each epoch
)

show_val_results(final_history, X_val, y_val, CLASSES)
show_test_results(final_model,   X_test, y_test, CLASSES)



Epoch 1/50


C:\Users\me\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 - 9s - 654ms/step - accuracy: 0.2500 - loss: 1.6655 - val_accuracy: 0.3237 - val_loss: 1.5587
Epoch 2/50
14/14 - 8s - 552ms/step - accuracy: 0.3624 - loss: 1.4655 - val_accuracy: 0.4589 - val_loss: 1.2886
Epoch 3/50
14/14 - 7s - 496ms/step - accuracy: 0.4702 - loss: 1.2913 - val_accuracy: 0.4444 - val_loss: 1.2251
Epoch 4/50
14/14 - 7s - 486ms/step - accuracy: 0.5505 - loss: 1.1409 - val_accuracy: 0.5990 - val_loss: 1.0896
Epoch 5/50
14/14 - 8s - 580ms/step - accuracy: 0.6044 - loss: 1.0402 - val_accuracy: 0.6763 - val_loss: 0.9640
Epoch 6/50
14/14 - 9s - 623ms/step - accuracy: 0.6972 - loss: 0.8846 - val_accuracy: 0.6329 - val_loss: 0.8799
Epoch 7/50
14/14 - 7s - 531ms/step - accuracy: 0.7087 - loss: 0.7798 - val_accuracy: 0.7150 - val_loss: 0.7185
Epoch 8/50
14/14 - 7s - 497ms/step - accuracy: 0.7798 - loss: 0.6407 - val_accuracy: 0.7826 - val_loss: 0.7304
Epoch 9/50
14/14 - 7s - 520ms/step - accuracy: 0.7982 - loss: 0.5602 - val_accuracy: 0.7633 - val_loss: 0.5993
Epoch 10/50


In [83]:
## saving the final model


teliko_modelo = "data/genres_transfer_model.h5"
final_model.save(teliko_modelo)
print(f"Model saved as {teliko_modelo}")

Model saved as data/genres_transfer_model.h5


In [84]:
### prep-split for transfer learning



import os
import numpy as np
import cv2

DATA_DIR = "data/transfer_set"
IMAGE_SIZE = (128, 128)
SEED = 42
NUM_CLASSES = 10

# list of genre folders
genres = sorted(os.listdir(DATA_DIR))
genre_to_idx = {genre: idx for idx, genre in enumerate(genres)}

X = []
y = []

for genre in genres:
    genre_folder = os.path.join(DATA_DIR, genre)
    for fname in os.listdir(genre_folder):
        if fname.lower().endswith('.png'):
            img_path = os.path.join(genre_folder, fname)

            # grayscale AGAIN
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMAGE_SIZE)
            img = img.astype(np.float32) / 255.0

            img = np.expand_dims(img, axis=-1)
            X.append(img)
            y.append(genre_to_idx[genre])

X = np.array(X)
y = np.array(y)

# split into train 70%, temp 30% >> then split temp into val & test

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=SEED,
    stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=SEED,
    stratify=y_temp
)

print("Shapes:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val:   {X_val.shape},   y_val:   {y_val.shape}")
print(f"  X_test:  {X_test.shape},  y_test:  {y_test.shape}")

Shapes:
  X_train: (699, 128, 128, 1), y_train: (699,)
  X_val:   (150, 128, 128, 1),   y_val:   (150,)
  X_test:  (150, 128, 128, 1),  y_test:  (150,)


In [85]:
### CNN for transfer learning



from tensorflow.keras.models import load_model, Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout

# the original model
base_model = load_model(teliko_modelo, compile=False)

# the output of the penultimate layer
x = base_model.layers[-2].output

# new output layer for 10 genres
new_output = Dense(10, activation='softmax')(x)

# Create new model
transfer_model = Model(inputs=base_model.layers[0].input, outputs=new_output)

# freezing earlier layers
for layer in transfer_model.layers:
    layer.trainable = True

transfer_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# training on the 10-genre dataset
transfer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 289ms/step - accuracy: 0.0878 - loss: 4.6430 - val_accuracy: 0.1000 - val_loss: 2.2983
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 274ms/step - accuracy: 0.1076 - loss: 2.2906 - val_accuracy: 0.2200 - val_loss: 2.2432
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 279ms/step - accuracy: 0.1880 - loss: 2.2437 - val_accuracy: 0.3333 - val_loss: 2.0886
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 295ms/step - accuracy: 0.2717 - loss: 2.0895 - val_accuracy: 0.4200 - val_loss: 1.8731
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 275ms/step - accuracy: 0.3120 - loss: 1.9384 - val_accuracy: 0.4333 - val_loss: 1.7306
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 285ms/step - accuracy: 0.3475 - loss: 1.8476 - val_accuracy: 0.4600 - val_loss: 1.6125
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 288ms/step - accuracy: 0.3819 - loss: 1.7440 - val_accuracy: 0.5267 - val_loss: 1.4797
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 282ms/step - accuracy: 0.4234 - loss: 1.6546 - val_accuracy: 0.

In [86]:
genres = sorted(os.listdir(DATA_DIR))
genre_to_idx = {g:i for i,g in enumerate(genres)}
trans_class_names = genres

show_test_results(transfer_model, X_test, y_test, trans_class_names)



=== TEST-SET RESULTS ===
Weighted F1 (test): 0.6193
              precision    recall  f1-score   support

       blues       0.57      0.53      0.55        15
   classical       0.87      0.87      0.87        15
     country       0.44      0.80      0.57        15
       disco       0.50      0.53      0.52        15
      hiphop       0.50      0.27      0.35        15
        jazz       0.75      0.80      0.77        15
       metal       0.92      0.73      0.81        15
         pop       0.71      0.80      0.75        15
      reggae       0.67      0.80      0.73        15
        rock       0.43      0.20      0.27        15

    accuracy                           0.63       150
   macro avg       0.64      0.63      0.62       150
weighted avg       0.64      0.63      0.62       150

Confusion matrix (test):
 [[ 8  0  4  1  0  2  0  0  0  0]
 [ 0 13  1  0  0  0  0  1  0  0]
 [ 1  0 12  0  0  1  0  0  0  1]
 [ 0  1  2  8  3  0  0  1  0  0]
 [ 0  0  0  5  4  0  0  2  4  

In [87]:
### RESULTS for the test's f1-score after 5 consecutive runs for *tranfer learning*


# run 1

# === TEST-SET RESULTS ===
# Weighted F1 (test): 0.6343
#               precision    recall  f1-score   support

#        blues       0.67      0.67      0.67        15
#    classical       0.81      0.87      0.84        15
#      country       0.42      0.73      0.54        15
#        disco       0.67      0.53      0.59        15
#       hiphop       0.56      0.60      0.58        15
#         jazz       0.63      0.80      0.71        15
#        metal       0.80      0.80      0.80        15
#          pop       0.69      0.60      0.64        15
#       reggae       0.73      0.53      0.62        15
#         rock       0.57      0.27      0.36        15

#     accuracy                           0.64       150
#    macro avg       0.66      0.64      0.63       150
# weighted avg       0.66      0.64      0.63       150

# Confusion matrix (test):
#  [[10  0  3  0  0  2  0  0  0  0]
#  [ 0 13  1  0  0  0  0  1  0  0]
#  [ 0  1 11  0  0  2  0  0  0  1]
#  [ 0  1  2  8  4  0  0  0  0  0]
#  [ 0  0  0  1  9  0  1  2  2  0]
#  [ 2  0  1  0  0 12  0  0  0  0]
#  [ 1  0  0  0  1  0 12  0  0  1]
#  [ 0  0  1  3  1  0  0  9  0  1]
#  [ 1  0  2  0  1  1  1  1  8  0]
#  [ 1  1  5  0  0  2  1  0  1  4]]


# run 2

# === TEST-SET RESULTS ===
# Weighted F1 (test): 0.6247
#               precision    recall  f1-score   support

#        blues       0.59      0.67      0.62        15
#    classical       0.81      0.87      0.84        15
#      country       0.44      0.53      0.48        15
#        disco       0.40      0.53      0.46        15
#       hiphop       0.56      0.60      0.58        15
#         jazz       0.85      0.73      0.79        15
#        metal       0.79      0.73      0.76        15
#          pop       0.64      0.60      0.62        15
#       reggae       0.89      0.53      0.67        15
#         rock       0.46      0.40      0.43        15

#     accuracy                           0.62       150
#    macro avg       0.64      0.62      0.62       150
# weighted avg       0.64      0.62      0.62       150

# Confusion matrix (test):
#  [[10  0  3  2  0  0  0  0  0  0]
#  [ 0 13  1  0  0  0  0  1  0  0]
#  [ 0  1  8  0  0  2  0  0  0  4]
#  [ 0  1  1  8  4  0  0  1  0  0]
#  [ 0  0  0  3  9  0  1  2  0  0]
#  [ 3  0  1  0  0 11  0  0  0  0]
#  [ 1  0  0  0  1  0 11  0  0  2]
#  [ 0  0  1  4  0  0  0  9  0  1]
#  [ 1  0  0  2  2  0  1  1  8  0]
#  [ 2  1  3  1  0  0  1  0  1  6]]

# run 3

# === TEST-SET RESULTS ===
# Weighted F1 (test): 0.6327
#               precision    recall  f1-score   support

#        blues       0.52      0.80      0.63        15
#    classical       0.81      0.87      0.84        15
#      country       0.64      0.60      0.62        15
#        disco       0.53      0.53      0.53        15
#       hiphop       0.57      0.53      0.55        15
#         jazz       0.62      0.87      0.72        15
#        metal       0.79      0.73      0.76        15
#          pop       0.67      0.53      0.59        15
#       reggae       0.69      0.60      0.64        15
#         rock       0.62      0.33      0.43        15

#     accuracy                           0.64       150
#    macro avg       0.65      0.64      0.63       150
# weighted avg       0.65      0.64      0.63       150

# Confusion matrix (test):
#  [[12  0  0  0  0  3  0  0  0  0]
#  [ 0 13  1  0  0  0  0  1  0  0]
#  [ 1  1  9  0  0  3  0  0  0  1]
#  [ 1  1  1  8  4  0  0  0  0  0]
#  [ 0  0  0  2  8  0  1  2  2  0]
#  [ 2  0  0  0  0 13  0  0  0  0]
#  [ 2  0  0  0  1  0 11  0  0  1]
#  [ 0  0  1  5  0  0  0  8  0  1]
#  [ 2  0  0  0  1  1  1  1  9  0]
#  [ 3  1  2  0  0  1  1  0  2  5]]

# run 4

# === TEST-SET RESULTS ===
# Weighted F1 (test): 0.6214
#               precision    recall  f1-score   support

#        blues       0.57      0.53      0.55        15
#    classical       0.82      0.93      0.88        15
#      country       0.45      0.60      0.51        15
#        disco       0.54      0.47      0.50        15
#       hiphop       0.56      0.60      0.58        15
#         jazz       0.71      0.80      0.75        15
#        metal       0.79      0.73      0.76        15
#          pop       0.67      0.67      0.67        15
#       reggae       0.53      0.60      0.56        15
#         rock       0.71      0.33      0.45        15

#     accuracy                           0.63       150
#    macro avg       0.63      0.63      0.62       150
# weighted avg       0.63      0.63      0.62       150

# Confusion matrix (test):
#  [[ 8  0  3  0  0  1  0  0  3  0]
#  [ 0 14  0  0  0  0  0  1  0  0]
#  [ 0  1  9  1  0  3  0  0  0  1]
#  [ 0  1  2  7  3  0  0  1  1  0]
#  [ 0  0  0  1  9  0  1  2  2  0]
#  [ 3  0  0  0  0 12  0  0  0  0]
#  [ 1  0  1  1  1  0 11  0  0  0]
#  [ 0  0  1  2  1  0  0 10  0  1]
#  [ 1  0  1  0  2  0  1  1  9  0]
#  [ 1  1  3  1  0  1  1  0  2  5]]

# run 5

# === TEST-SET RESULTS ===
# Weighted F1 (test): 0.6196
#               precision    recall  f1-score   support

#        blues       0.67      0.53      0.59        15
#    classical       0.74      0.93      0.82        15
#      country       0.64      0.47      0.54        15
#        disco       0.44      0.47      0.45        15
#       hiphop       0.53      0.60      0.56        15
#         jazz       0.67      0.93      0.78        15
#        metal       0.79      0.73      0.76        15
#          pop       0.53      0.60      0.56        15
#       reggae       0.75      0.60      0.67        15
#         rock       0.55      0.40      0.46        15

#     accuracy                           0.63       150
#    macro avg       0.63      0.63      0.62       150
# weighted avg       0.63      0.63      0.62       150

# Confusion matrix (test):
#  [[ 8  0  1  2  0  3  0  0  1  0]
#  [ 0 14  0  0  0  0  0  1  0  0]
#  [ 0  2  7  1  0  2  0  1  0  2]
#  [ 0  1  1  7  4  0  0  2  0  0]
#  [ 0  0  0  3  9  0  1  2  0  0]
#  [ 1  0  0  0  0 14  0  0  0  0]
#  [ 1  0  0  0  1  0 11  0  0  2]
#  [ 0  0  1  3  1  0  0  9  0  1]
#  [ 1  0  0  0  2  0  1  2  9  0]
#  [ 1  2  1  0  0  2  1  0  2  6]]




In [88]:
#printing the accuracy and val accuracy of the final weights
train_loss, train_acc = transfer_model.evaluate(X_train, y_train, verbose=0)
val_loss, val_acc = transfer_model.evaluate(X_val, y_val, verbose=0)
print(f"Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")

# printing the per class distribution in x and y
from collections import Counter
def print_class_distribution(y, split_name):
    counts = Counter(y)
    total = len(y)
    print(f"Class distribution in {split_name}:")

    for cls, count in sorted(counts.items()):
        print(f"  Class {cls}: {count} samples, {count / total:.2%}")
    print(f"  Total samples: {total}\n")
print_class_distribution(y_train, 'Training Set')
print_class_distribution(y_val, 'Validation Set')


Train Accuracy: 0.9342, Validation Accuracy: 0.6867
Class distribution in Training Set:
  Class 0: 70 samples, 10.01%
  Class 1: 70 samples, 10.01%
  Class 2: 70 samples, 10.01%
  Class 3: 70 samples, 10.01%
  Class 4: 70 samples, 10.01%
  Class 5: 69 samples, 9.87%
  Class 6: 70 samples, 10.01%
  Class 7: 70 samples, 10.01%
  Class 8: 70 samples, 10.01%
  Class 9: 70 samples, 10.01%
  Total samples: 699

Class distribution in Validation Set:
  Class 0: 15 samples, 10.00%
  Class 1: 15 samples, 10.00%
  Class 2: 15 samples, 10.00%
  Class 3: 15 samples, 10.00%
  Class 4: 15 samples, 10.00%
  Class 5: 15 samples, 10.00%
  Class 6: 15 samples, 10.00%
  Class 7: 15 samples, 10.00%
  Class 8: 15 samples, 10.00%
  Class 9: 15 samples, 10.00%
  Total samples: 150

